In [171]:
import json
from pymongo import MongoClient
from bson.objectid import ObjectId
import numpy as np
import random
import re
#!pip install --upgrade --quiet  rank_bm25
from rank_bm25 import BM25Okapi
import os

import sys
sys.path.append("../../../")

from utils.helper_functions import clean_title
from BM25_retriever import BM25Retriever
from helper_functions import remove_punctuation, lemmatization#, remove_stopwords

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
client = MongoClient("mongodb://mongo_db:27017")
db = client["SAVIA"]
coll_demetra = db["leggiRegionali"]
print(sorted(db.list_collection_names()))

['attiAssembleaLegislativa', 'attiComuneFerrara', 'attiComuneModena', 'attiComuneParma', 'attiComunePiacenza', 'attiComuneRavenna', 'attiComuneReggio', 'attiComuneRimini', 'attiGiuntaRegionale', 'chunks_leggiRegionali', 'leggiRegionali', 'leggiRegionaliOld']


### Create corpus

In [163]:
#list(doc['articoli'].values())[0]

In [164]:
#list(doc['articoli'].values())

In [131]:
list_all_laws = []
corpus = []

for ind, doc in enumerate(coll_demetra.find()[0:]):
    titolo_orig = doc['titolo']
#    titolo = titolo_orig
    titolo = clean_title(titolo_orig).lower()
    out_dict = {"titolo": titolo, "legge": doc["legge"], 
                          "testo": doc['testo'].replace("\n", " ").lower(),#, "numero": doc["numero"],
                        "legge": doc['legge'], "numeroAnno": doc["numeroAnno"]}

    if 'articoli' in doc.keys():
        articoli = list(doc['articoli'].values())
        articoli_str = " ".join([x for x in articoli[0:2]]).lower()
        
        out_dict["articoli"] = articoli_str

    list_all_laws.append(out_dict)

    corpus.append(" ".join(list(out_dict.values())))
    
#corpus_raw = [x['testo'] for x in list_all_laws]
#corpus = corpus_raw
#corpus = [remove_punctuation(x) for x in corpus_raw]
#corpus = [remove_stopwords(x) for x in corpus]
#corpus = [lemmatization(x) for x in corpus]

#corpus = [x['titolo'] for x in list_all_laws]
#corpus = [x['titolo'] + " " + x['legge'] + " " + x['numeroAnno'] + " " + x['testo'] for x in list_all_laws]

#corpus = [x['titolo'] + " " + x['legge'] + " " + x['numeroAnno'] + " " + x['articoli'][0] +
#          " " + x['articoli'][1] 
#          for x in list_all_laws]

#keys = ["titolo", "legge", "testo", "numero legge"]
#corpus = []
#for item in list_all_laws[0:]:
#    out_str = ""
#    for k in keys:
#        out_str += " " + item[k]
#    corpus.append(out_str.strip())

### Read question from IB dataset

In [139]:
retriever = BM25Retriever(corpus, top_n = 30, threshold = None)
#retriever_raw = BM25Retriever(corpus_raw, top_n = 10, threshold = None)

In [167]:
dataset_folder = "../../../../datasets/instruction_datasets/instruction_dataset_v2"

with open(os.path.join(dataset_folder, "IB_dataset_train.json"), "r") as f:
    IB_dataset_train = json.load(f)

with open(os.path.join(dataset_folder, "IB_dataset_test.json"), "r") as f:
    IB_dataset_test = json.load(f)

In [238]:
ind = random.randint(0, len(IB_dataset_train) - 1)
#ind = 825
print(ind)
question = IB_dataset_train[ind]['question']
print(question)
#print("---------------------------------")
#print(IB_dataset[ind]['answer'])

937
Sulla legge n. 07 del 1976 quanti atti attuativi sono stati emanati?


In [239]:
#question = 'Quali valutazioni sono stati condotte sulla l.r. "LEGGE REGIONALE 03 ottobre 2023, n. 12" sullo sviluppo dell\'economia urbana'

### Retrieve with keywords

In [240]:
for ind, doc in enumerate(coll_demetra.find()[0:]):
    keywords = doc['keywords']
    for keyword in keywords[0:]:
#        print(keyword)
        if re.search(keyword, question, re.IGNORECASE):
            print("found", doc['legge'])
            break

### Retrieve with BM25

In [160]:
top_n_inds, top_n_sim = retriever.retrieve(question)

for ind_sim, ind in enumerate(top_n_inds):
    print(str(ind_sim) + " - score " + str(top_n_sim[ind_sim])) 
    print("titolo:", list_all_laws[ind]['titolo'].upper())
    print("legge: {}".format(list_all_laws[ind]['legge'].upper()))
          
    print("--------------")

0 - score 29.79908
titolo: RATIFICA DEL PROTOCOLLO D'INTESA TRA LA REGIONE EMILIA-ROMAGNA, L'AUTORITÀ DI BACINO DEL FIUME PO, LA REGIONE LOMBARDIA, LA REGIONE PIEMONTE, LA REGIONE VENETO PER UNA GESTIONE SOSTENIBILE E UNITARIA DELLA PESCA E PER LA TUTELA DEL PATRIMONIO ITTICO NEL FIUME PO
legge: LEGGE REGIONALE 03 APRILE 2018, N. 3
--------------
1 - score 28.26559
titolo: NUOVI INTERVENTI DELLA REGIONE PER IL CONTROLLO E LA PREVENZIONE DEGLI INQUINAMENTI ATMOSFERICI E IDRICI(1)(2)(3)
legge: LEGGE REGIONALE 22 GENNAIO 1980, N. 6
--------------
2 - score 26.29624
titolo: DISCIPLINA REGIONALE SULLA TUTELA E L’USO DEL TERRITORIO
legge: LEGGE REGIONALE 21 DICEMBRE 2017, N. 24
--------------
3 - score 25.87937
titolo: MISURE PER L'ATTUAZIONE DELLA LEGGE 28 MAGGIO 2021, N. 84 , CONCERNENTE IL DISTACCO DEI COMUNI DI MONTECOPIOLO E SASSOFELTRIO DALLA REGIONE MARCHE E LORO AGGREGAZIONE ALLA REGIONE EMILIA-ROMAGNA
legge: LEGGE REGIONALE 26 NOVEMBRE 2021, N. 18
--------------
4 - score 24.80775
t

In [190]:
res = coll_demetra.find_one({"legge": "LEGGE REGIONALE 02 luglio 2019, n. 9"})
res['keywords']

['l.r. 02 luglio 2019, n. 9',
 'l.r. 02 luglio 2019, n.9',
 'l.r. 02 luglio 2019, num. 9',
 'l.r. 02 luglio 2019, num.9',
 'l.r. 02-07-19, n. 9',
 'l.r. 02-07-19, n.9',
 'l.r. 02-07-19, num. 9',
 'l.r. 02-07-19, num.9',
 'l.r. 02-07-2019, n. 9',
 'l.r. 02-7-19, n. 9',
 'l.r. 02-7-2019, n.9',
 'l.r. 02/07/19, n. 9',
 'l.r. 02/07/19, n.9',
 'l.r. 02/07/19, num. 9',
 'l.r. 02/07/19, num.9',
 'l.r. 02/07/2019, n. 9',
 'l.r. 02/7/19, n. 9',
 'l.r. 02/7/2019, n.9',
 'l.r. 9 del 02 luglio 2019',
 'l.r. 9/2019',
 'l.r. del 02 luglio 2019 n. 9',
 'l.r. del 02 luglio 2019 n.9',
 'l.r. del 02 luglio 2019 num. 9',
 'l.r. del 02 luglio 2019 num.9',
 'l.r. n. 9 del 02-07-19',
 'l.r. n. 9 del 02-07-2019',
 'l.r. n. 9 del 02/07/19',
 'l.r. n. 9 del 02/07/2019',
 'l.r. n. 9 del2019',
 'l.r. n. 9/19',
 'l.r. n. 9/2019',
 'l.r. n.9 del 02-07-19',
 'l.r. n.9 del 02-07-2019',
 'l.r. n.9 del 02/07/19',
 'l.r. n.9 del 02/07/2019',
 'l.r. n.9 del2019',
 'l.r. n.9/19',
 'l.r. n.9/2019',
 'l.r. num. 9 del 02-07

In [ ]:
legge regionale num. 9 del 2019

In [166]:
keywords

['l.r. 1 luglio 2024, n. 11',
 'l.r. 1 luglio 2024, n.11',
 'l.r. 1 luglio 2024, num. 11',
 'l.r. 1 luglio 2024, num.11',
 'l.r. 1-07-2024, n. 11',
 'l.r. 1-07-24, n. 11',
 'l.r. 1-07-24, n.11',
 'l.r. 1-07-24, num. 11',
 'l.r. 1-07-24, num.11',
 'l.r. 1-7-2024, n.11',
 'l.r. 1-7-24, n. 11',
 'l.r. 1/07/2024, n. 11',
 'l.r. 1/07/24, n. 11',
 'l.r. 1/07/24, n.11',
 'l.r. 1/07/24, num. 11',
 'l.r. 1/07/24, num.11',
 'l.r. 1/7/2024, n.11',
 'l.r. 1/7/24, n. 11',
 'l.r. 11 del 1 luglio 2024',
 'l.r. 11/2024',
 'l.r. del 1 luglio 2024 n. 11',
 'l.r. del 1 luglio 2024 n.11',
 'l.r. del 1 luglio 2024 num. 11',
 'l.r. del 1 luglio 2024 num.11',
 'l.r. n. 11 del 1-07-2024',
 'l.r. n. 11 del 1-07-24',
 'l.r. n. 11 del 1-7-2024',
 'l.r. n. 11 del 1-7-24',
 'l.r. n. 11 del 1/07/2024',
 'l.r. n. 11 del 1/07/24',
 'l.r. n. 11 del 1/7/2024',
 'l.r. n. 11 del 1/7/24',
 'l.r. n. 11/2024',
 'l.r. n. 11/24',
 'l.r. n.11 del 1-07-2024',
 'l.r. n.11 del 1-07-24',
 'l.r. n.11 del 1/07/2024',
 'l.r. n.11 del

In [ ]:
#corpus_raw